<a href="https://colab.research.google.com/github/SergeyKamenshchikov/Sksummy/blob/master/paraphrase_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install

In [ ]:
!pip3 install rouge
!pip3 install xlsxwriter
!pip3 install xlrd

!pip3 install transformers
!pip3 install sentencepiece

!pip3 install textstat

import nltk
nltk.download('punkt')

import

In [ ]:
import pandas
import json
import numpy

from nltk.tokenize import sent_tokenize, word_tokenize

import random

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

from rouge import Rouge
rouge = Rouge()

import textstat

get wiki corpus texts

In [ ]:
#download wiki corpus by link
!wget -O wiki_corpus https://drive.google.com/u/2/uc?id=1HvzyuQhM3VGMe5yiil-0vYk0fqvZM44F&export=download

In [ ]:
#open corpus file and add texts to list
df = pandas.read_excel('/content/wiki_corpus')

wiki_corpus_texts=df[0].tolist()

df

get arxiv corpus

In [ ]:
#download arxiv corpus
!wget --no-check-certificate -r -O arxiv_corpus.zip https://drive.google.com/u/0/uc?id=1b3rmCSIoh6VhD4HKWjI4HOW-cSwcwbeC&export=download

In [ ]:
#unzip corpus - may take long due to big size
!unzip /content/arxiv_corpus.zip

In [ ]:
#open test file
f=open(r'/content/arxiv-dataset/test.txt','r')
corpus_lines = f.readlines()

In [ ]:
#add text to list
arxiv_corpus_texts=[]

for line in corpus_lines:
    
    #line to json
    data = json.loads(line)
    
    #get text of article
    text=data['article_text']
    text=' '.join(text)
     
    arxiv_corpus_texts.append(text)

In [ ]:
#normalize data

target_length = 100

arxiv_target_articles_texts=[ text for text in arxiv_corpus_texts if len(sent_tokenize(text))>target_length-1]

print('Num of articles of desired length:',len(arxiv_target_articles_texts))

arxiv_balanced_target_articles_texts=[' '.join(sent_tokenize(text)[:target_length]) for text in arxiv_target_articles_texts]


Num of articles of desired length: 5740


Parameters

In [ ]:
#paraphrase model
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
#testing parameters
MAX_INPUT_LEN_K=1

MAX_OUTPUT_LEN_K=1
MIN_OUTPUT_LEN_K=1

BEAMS=3

COPRUS_SAMPLES=5

Functions

In [ ]:
def readability_ratio(text1,text2):
  text1_syll=textstat.syllable_count(text1)
  text2_syll=textstat.syllable_count(text2)
  return(text2_syll/text1_syll*100)

def pegasus_count_tokens(text):
  #count number of tokens by pegasus_tokenizer
  return len(tokenizer(text)['input_ids'])

def paraphrase_sentence(input_text):
  input_length = pegasus_count_tokens(input_text)
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest', max_length=round(input_length*MAX_INPUT_LEN_K,0), return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=round(input_length*MAX_OUTPUT_LEN_K,0),min_length=round(input_length*MIN_OUTPUT_LEN_K,0), num_beams=BEAMS, num_return_sequences=1, temperature=1.5 )
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

def paraphrase_text(text):
  sentences=sent_tokenize(text)
  new_sentences=[]
  for sentence in sentences:
    try:
      new_sentence=paraphrase_sentence(sentence)[0]
      new_sentences.append(new_sentence)
    except IndexError:
      print('Error: Sentence is too long!')
      new_sentences.append(sentence)

  new_text=' '.join(new_sentences)
  return new_text

def evaluate(corpus_texts):
  rouge_L_scores=[]
  readability_scores=[]

  corpus_texts=random.sample(corpus_texts,COPRUS_SAMPLES)

  i=0
  for corpus_text in corpus_texts:
    new_text=paraphrase_text(corpus_text)

    rouge_L_scores.append(get_rouge_L_f(corpus_text,new_text))
    readability_scores.append(readability_ratio(corpus_text,new_text))

    #progress
    i=i+1
    print(i,' Texts processed')

  #print results
  print('\n','-----')
  print('Number of Texts: ', COPRUS_SAMPLES,'\n')
  print('ROUGE L median: ', numpy.median(rouge_L_scores))
  print('ROUGE L deviation: ',numpy.std(rouge_L_scores),'\n')
  print('Readability median: ', numpy.median(readability_scores))
  print('Readability deviation: ',numpy.std(readability_scores))

def get_rouge_L_f(text1,text2):
    scores = rouge.get_scores(text1, text2)
    score_f=scores[0]['rouge-l']['f']
    return(score_f)


Testing paraphrase

In [ ]:
%%time
evaluate(wiki_corpus_texts)

Error: Sentence is too long!
Error: Sentence is too long!
1  Texts processed
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
2  Texts processed
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
3  Texts processed
Error: Sentence is too long!
4  Texts processed
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
5  Texts processed

 -----
Number of Texts:  5 

ROUGE L median:  0.7440982008600414
ROUGE L deviation:  0.060311375467823926 

Readability median:  94.66212379329926
Readability deviation:  2.4900596922420073
CPU times: user 17min 24s, sys: 8.46 s, total: 17min 33s
Wall time: 17min 35s


In [ ]:
%%time
evaluate(arxiv_corpus_texts)

Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentence is too long!
Error: Sentenc